## WeGo Route 50 Data TSP Segmentation

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
wego = pd.read_csv("../data/wego_contextual_segmentation.csv")

### 1. Assigning TSP Phase to Each Trip

These phases include:

- `Pre-TSP` – before any TSP was active (... - 02.03.2025 12:00 pm)

- `TSP Unconditional 1` – full TSP activation (02.03.2025 12:00 pm - 02.10.2025 12:00 pm)

- `TSP Schedule-Conditional` – TSP activates only if bus is late more than 2 min (02.10.2025 12:00 pm - 04.28.2025 12:00 pm)

- `TSP Off` – TSP deactivated (04.28.2025 12:00 pm - 05.05.2025 12:00 pm)

- `TSP Unconditional 2` – full TSP activation (05.12.2025 12:00 pm - 05.05.2025 12:00 pm)

- `TSP Headway-Conditional` – TSP activates if headway deviation exceeds 120% (05.05.2025 12:00 pm - ...)

In [3]:
wego['DATETIME'] = pd.to_datetime(wego['DATE'].astype(str) + ' ' + wego['SCHEDULED_TIME'].astype(str), errors='coerce')

def assign_tsp_phase(dt):
    if pd.isna(dt):
        return ('Pre-TSP')
    if dt < datetime(2025, 2, 3, 12):
        return ('Pre-TSP')
    elif dt < datetime(2025, 2, 10, 12):
        return ('TSP Unconditional 1')
    elif dt < datetime(2025, 4, 28, 12):
        return ('TSP Schedule-Conditional')
    elif dt < datetime(2025, 5, 5, 12):
        return ('TSP Off')
    elif dt < datetime(2025, 5, 12, 12):
        return ('TSP Unconditional 2')
    else:
        return ('TSP Headway-Conditional')

wego['TSP_PHASE_NAME'] = wego['DATETIME'].apply(assign_tsp_phase)

### 2. Assigning TSP_ACTIVE Flag
This function creates a new column TSP_ACTIVE which indicates whether Transit Signal Priority (TSP) was actively applied to a specific timepoint record ('Y' or 'N').

In [4]:
def assign_tsp_active(row):
    phase = row['TSP_PHASE_NAME']

    timepoint = row['TIME_POINT_ABBR']
    
    tsp_timepoints = ['WHBG', 'CH46', '28&CHARL', 'MCC5_1']
    if timepoint not in tsp_timepoints:
        return 'N'
    
    if phase == 'Pre-TSP' or phase == 'TSP Off':
        return 'N'
    
    elif phase in ['TSP Unconditional 1', 'TSP Unconditional 2']:
        return 'Y'
    
    elif phase == 'TSP Schedule-Conditional':
        delay_minutes = row['ADHERENCE']
        
        if delay_minutes < -2:
            return 'Y'  
        else: 'N'

    elif phase == 'TSP Headway-Conditional':
        hdwy = row['HDWY_DEV']
        
        if hdwy > 1.2:
            return 'Y'  
        else: 'N'
            
    else:
        return 'N'

wego['TSP_ACTIVE'] = wego.apply(assign_tsp_active, axis=1)

In [7]:
wego.head()

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,DWELL_IN_MINS,TIME_OF_DAY,SEASON,IS_HOLIDAY,SCHEDULED_MINUTES,IN_SCHOOL_ZONE_TIME,DAY_TYPE,DATETIME,TSP_PHASE_NAME,TSP_ACTIVE
0,120230101,3,93549161,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,8.133333,Midday,Winter,False,334,N,Sunday,2023-01-01 05:34:00,Pre-TSP,N
1,120230101,3,93549162,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.000000,Midday,Winter,False,340,N,Sunday,2023-01-01 05:40:00,Pre-TSP,N
2,120230101,3,93549163,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.000000,Midday,Winter,False,347,N,Sunday,2023-01-01 05:47:00,Pre-TSP,N
3,120230101,3,93549164,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.000000,Midday,Winter,False,350,N,Sunday,2023-01-01 05:50:00,Pre-TSP,N
4,120230101,3,93549165,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,2.150000,Midday,Winter,False,354,N,Sunday,2023-01-01 05:54:00,Pre-TSP,N


### 3. Save the cleaned and enriched dataset

In [8]:
wego.to_csv("../data/wego_tsp_assigned.csv", index=False)

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,DWELL_IN_MINS,TIME_OF_DAY,SEASON,IS_HOLIDAY,SCHEDULED_MINUTES,IN_SCHOOL_ZONE_TIME,DAY_TYPE,DATETIME,TSP_PHASE_NAME,TSP_ACTIVE
0,120230101,3,93549161,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,8.133333,Midday,Winter,False,334,N,Sunday,2023-01-01 05:34:00,Pre-TSP,N
1,120230101,3,93549162,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.000000,Midday,Winter,False,340,N,Sunday,2023-01-01 05:40:00,Pre-TSP,N
2,120230101,3,93549163,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.000000,Midday,Winter,False,347,N,Sunday,2023-01-01 05:47:00,Pre-TSP,N
3,120230101,3,93549164,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.000000,Midday,Winter,False,350,N,Sunday,2023-01-01 05:50:00,Pre-TSP,N
4,120230101,3,93549165,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,2.150000,Midday,Winter,False,354,N,Sunday,2023-01-01 05:54:00,Pre-TSP,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618993,120250512,1,119531049,2025-05-12,50,8401,3077,432121,0,FROM DOWNTOWN,...,4.500000,AM Peak,Spring,False,460,Y,Weekday,2025-05-12 07:40:00,TSP Unconditional 2,N
618994,120250512,1,119531638,2025-05-12,50,9302,3246,432353,0,TO DOWNTOWN,...,27.283333,PM Peak,Spring,False,905,Y,Weekday,2025-05-12 15:05:00,TSP Headway-Conditional,N
618995,120250512,1,119531639,2025-05-12,50,9302,3246,432353,0,TO DOWNTOWN,...,0.133333,PM Peak,Spring,False,918,Y,Weekday,2025-05-12 15:18:00,TSP Headway-Conditional,N
618996,120250512,1,119531770,2025-05-12,50,9950,2448,432387,0,TO DOWNTOWN,...,8.166666,PM Peak,Spring,False,920,Y,Weekday,2025-05-12 15:20:00,TSP Headway-Conditional,N


In [11]:
cln_wego = wego[(wego['OVERLOAD_ID'] == 0) &
(wego['TRIP_EDGE'] == 0) &
(wego['STOP_CANCELLED'] == 0) &
(~wego['TIME_POINT_ABBR'].str.contains('HLWD', na=False))]

In [13]:
non_nan_values = cln_wego['SCHEDULED_HDWY'].dropna()

In [16]:
non_null_headway = cln_wego[['DATE', 'DAY_TYPE', 'TIME_OF_DAY', 'SEASON', 'IN_SCHOOL_ZONE_TIME', 'IS_HOLIDAY', 'OPERATOR', 'TIME_POINT_ABBR', 'ROUTE_DIRECTION_NAME', 'SCHEDULED_TIME', 'ACTUAL_ARRIVAL_TIME', 'ADHERENCE', 'DWELL_IN_MINS', 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 'HDWY_DEV', 'DATETIME', 'TSP_PHASE_NAME',	'TSP_ACTIVE']].dropna()
non_null_headway 

,DATE,DAY_TYPE,TIME_OF_DAY,SEASON,IN_SCHOOL_ZONE_TIME,IS_HOLIDAY,OPERATOR,TIME_POINT_ABBR,ROUTE_DIRECTION_NAME,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ADHERENCE,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,DATETIME,TSP_PHASE_NAME,TSP_ACTIVE
14,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,WHBG,TO DOWNTOWN,07:17:00,07:20:48,-5.483333,1.683333,30.0,34.016666,4.016666,2023-01-01 07:17:00,Pre-TSP,N
15,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,CH46,TO DOWNTOWN,07:20:00,07:24:18,-4.300000,0.000000,30.0,34.066666,4.066666,2023-01-01 07:20:00,Pre-TSP,N
16,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,28&CHARL,TO DOWNTOWN,07:24:00,07:26:41,-2.683333,0.000000,30.0,32.516666,2.516666,2023-01-01 07:24:00,Pre-TSP,N
19,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,28&CHARL,FROM DOWNTOWN,07:55:00,07:56:33,-1.550000,0.000000,30.0,31.233333,1.233333,2023-01-01 07:55:00,Pre-TSP,N
20,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,CH46,FROM DOWNTOWN,07:59:00,07:59:04,-2.533333,2.466666,30.0,32.166666,2.166666,2023-01-01 07:59:00,Pre-TSP,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618944,2025-05-12,Weekday,PM Peak,Spring,Y,False,2706,28&CHARL,TO DOWNTOWN,16:18:00,16:27:13,-10.983333,1.766666,15.0,24.333333,9.333333,2025-05-12 16:18:00,TSP Headway-Conditional,Y
618971,2025-05-12,Weekday,PM Peak,Spring,Y,False,754,28&CHARL,FROM DOWNTOWN,15:42:00,15:45:15,-3.250000,0.000000,15.0,17.400000,2.400000,2025-05-12 15:42:00,TSP Headway-Conditional,Y
618972,2025-05-12,Weekday,PM Peak,Spring,Y,False,754,CH46,FROM DOWNTOWN,15:47:00,15:50:35,-3.583333,0.000000,15.0,17.100000,2.100000,2025-05-12 15:47:00,TSP Headway-Conditional,Y
618973,2025-05-12,Weekday,PM Peak,Spring,Y,False,754,WHBG,FROM DOWNTOWN,15:52:00,15:56:40,-4.666666,0.000000,15.0,16.600000,1.600000,2025-05-12 15:52:00,TSP Headway-Conditional,Y


In [17]:
non_null_headway['HDWY_DEV_PCT'] = (non_null_headway['HDWY_DEV'] / non_null_headway['SCHEDULED_HDWY']) * 100
non_null_headway.sort_values(['HDWY_DEV'])

,DATE,DAY_TYPE,TIME_OF_DAY,SEASON,IN_SCHOOL_ZONE_TIME,IS_HOLIDAY,OPERATOR,TIME_POINT_ABBR,ROUTE_DIRECTION_NAME,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ADHERENCE,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,DATETIME,TSP_PHASE_NAME,TSP_ACTIVE,HDWY_DEV_PCT
155958,2023-08-10,Weekday,AM Peak,Summer,N,False,1266,WHBG,FROM DOWNTOWN,06:32:00,06:54:56,-22.933333,0.000000,35.0,1.750000,-33.250000,2023-08-10 06:32:00,Pre-TSP,N,-95.000000
354702,2024-05-11,Saturday,Overnight,Spring,N,False,1742,28&CHARL,TO DOWNTOWN,23:54:00,23:53:08,-1.066666,1.933333,30.0,0.016666,-29.983334,2024-05-11 23:54:00,Pre-TSP,N,-99.944447
354263,2024-05-11,Saturday,Evening,Spring,N,False,3102,WHBG,FROM DOWNTOWN,22:01:00,22:06:28,-7.866666,2.400000,30.0,0.233333,-29.766667,2024-05-11 22:01:00,Pre-TSP,N,-99.222223
134741,2023-07-11,Weekday,Evening,Summer,N,False,1623,WHBG,TO DOWNTOWN,19:47:00,20:21:16,-34.266666,0.000000,31.0,1.733333,-29.266667,2023-07-11 19:47:00,Pre-TSP,N,-94.408603
155957,2023-08-10,Weekday,AM Peak,Summer,N,False,1266,CH46,FROM DOWNTOWN,06:28:00,06:35:30,-25.083333,17.583333,35.0,5.816666,-29.183334,2023-08-10 06:28:00,Pre-TSP,N,-83.380954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284054,2024-02-05,Weekday,Evening,Winter,N,False,304,CH46,TO DOWNTOWN,19:20:00,19:45:57,-25.950000,0.000000,30.0,129.366666,99.366666,2024-02-05 19:20:00,Pre-TSP,N,331.222220
284053,2024-02-05,Weekday,Evening,Winter,N,False,304,WHBG,TO DOWNTOWN,19:16:00,19:41:02,-26.900000,1.866666,30.0,130.233333,100.233333,2024-02-05 19:16:00,Pre-TSP,N,334.111110
269020,2024-01-15,Saturday,Evening,Winter,N,True,907,WHBG,TO DOWNTOWN,19:47:00,19:47:04,-0.066666,0.000000,30.0,139.300000,109.300000,2024-01-15 19:47:00,Pre-TSP,N,364.333333
71119,2023-04-11,Weekday,Overnight,Spring,N,False,304,28&CHARL,TO DOWNTOWN,23:54:00,23:54:13,-0.216666,0.000000,30.0,146.500000,116.500000,2023-04-11 23:54:00,Pre-TSP,N,388.333333


In [18]:
non_null_headway[(non_null_headway['ADHERENCE'] < -20)].sort_values('ADHERENCE')

,DATE,DAY_TYPE,TIME_OF_DAY,SEASON,IN_SCHOOL_ZONE_TIME,IS_HOLIDAY,OPERATOR,TIME_POINT_ABBR,ROUTE_DIRECTION_NAME,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ADHERENCE,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,DATETIME,TSP_PHASE_NAME,TSP_ACTIVE,HDWY_DEV_PCT
434647,2024-08-30,Weekday,PM Peak,Summer,Y,False,3339,CH46,FROM DOWNTOWN,16:02:00,17:14:22,-451.866666,379.500000,15.0,36.716666,21.716666,2024-08-30 16:02:00,Pre-TSP,N,144.777773
583793,2025-03-25,Weekday,PM Peak,Spring,N,False,3296,WHBG,FROM DOWNTOWN,18:33:00,23:00:29,-267.483333,0.000000,15.0,29.533333,14.533333,2025-03-25 18:33:00,TSP Schedule-Conditional,Y,96.888887
433464,2024-08-29,Weekday,Midday,Summer,N,False,1705,CH46,TO DOWNTOWN,13:49:00,14:05:18,-236.733333,220.433333,14.0,11.750000,-2.250000,2024-08-29 13:49:00,Pre-TSP,N,-16.071429
582402,2025-03-24,Weekday,Midday,Spring,N,False,2094,28&CHARL,TO DOWNTOWN,09:24:00,12:24:24,-192.233333,11.833333,15.0,24.350000,9.350000,2025-03-24 09:24:00,TSP Schedule-Conditional,Y,62.333333
582400,2025-03-24,Weekday,Midday,Spring,N,False,2094,WHBG,TO DOWNTOWN,09:16:00,12:15:58,-179.966666,0.000000,14.0,13.733333,-0.266667,2025-03-24 09:16:00,TSP Schedule-Conditional,Y,-1.904764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456958,2024-09-30,Weekday,Midday,Fall,N,False,2832,CH46,TO DOWNTOWN,13:05:00,13:25:02,-20.033333,0.000000,15.0,36.250000,21.250000,2024-09-30 13:05:00,Pre-TSP,N,141.666667
241254,2023-12-06,Weekday,PM Peak,Winter,N,False,3142,WHBG,FROM DOWNTOWN,17:37:00,17:56:12,-20.016666,0.816666,15.0,10.100000,-4.900000,2023-12-06 17:37:00,Pre-TSP,N,-32.666667
16458,2023-01-24,Weekday,PM Peak,Winter,N,False,2109,WHBG,FROM DOWNTOWN,17:34:00,17:54:01,-20.016666,0.000000,12.0,31.316666,19.316666,2023-01-24 17:34:00,Pre-TSP,N,160.972217
78010,2023-04-21,Weekday,PM Peak,Spring,N,False,1277,WHBG,FROM DOWNTOWN,16:37:00,16:57:01,-20.016666,0.000000,15.0,29.800000,14.800000,2023-04-21 16:37:00,Pre-TSP,N,98.666667


In [27]:
non_null_headway

,DATE,DAY_TYPE,TIME_OF_DAY,SEASON,IN_SCHOOL_ZONE_TIME,IS_HOLIDAY,OPERATOR,TIME_POINT_ABBR,ROUTE_DIRECTION_NAME,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ADHERENCE,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,DATETIME,TSP_PHASE_NAME,TSP_ACTIVE,HDWY_DEV_PCT
14,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,WHBG,TO DOWNTOWN,07:17:00,07:20:48,-5.483333,1.683333,30.0,34.016666,4.016666,2023-01-01 07:17:00,Pre-TSP,Y,13.388887
15,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,CH46,TO DOWNTOWN,07:20:00,07:24:18,-4.300000,0.000000,30.0,34.066666,4.066666,2023-01-01 07:20:00,Pre-TSP,Y,13.555553
16,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,28&CHARL,TO DOWNTOWN,07:24:00,07:26:41,-2.683333,0.000000,30.0,32.516666,2.516666,2023-01-01 07:24:00,Pre-TSP,Y,8.388887
19,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,28&CHARL,FROM DOWNTOWN,07:55:00,07:56:33,-1.550000,0.000000,30.0,31.233333,1.233333,2023-01-01 07:55:00,Pre-TSP,Y,4.111110
20,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,CH46,FROM DOWNTOWN,07:59:00,07:59:04,-2.533333,2.466666,30.0,32.166666,2.166666,2023-01-01 07:59:00,Pre-TSP,Y,7.222220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618944,2025-05-12,Weekday,PM Peak,Spring,Y,False,2706,28&CHARL,TO DOWNTOWN,16:18:00,16:27:13,-10.983333,1.766666,15.0,24.333333,9.333333,2025-05-12 16:18:00,TSP Headway-Conditional,Y,62.222220
618971,2025-05-12,Weekday,PM Peak,Spring,Y,False,754,28&CHARL,FROM DOWNTOWN,15:42:00,15:45:15,-3.250000,0.000000,15.0,17.400000,2.400000,2025-05-12 15:42:00,TSP Headway-Conditional,Y,16.000000
618972,2025-05-12,Weekday,PM Peak,Spring,Y,False,754,CH46,FROM DOWNTOWN,15:47:00,15:50:35,-3.583333,0.000000,15.0,17.100000,2.100000,2025-05-12 15:47:00,TSP Headway-Conditional,Y,14.000000
618973,2025-05-12,Weekday,PM Peak,Spring,Y,False,754,WHBG,FROM DOWNTOWN,15:52:00,15:56:40,-4.666666,0.000000,15.0,16.600000,1.600000,2025-05-12 15:52:00,TSP Headway-Conditional,Y,10.666667
